In [1]:
import numpy as np
import pandas as pd
import os
from utils.read_EMR import read_EMR

In [2]:
folder_path="dataset"
all_Labels,all_EMR_data,all_Name_files=read_EMR(folder_path)

Cột	Ý nghĩa
Gestation	Tuổi thai tại lúc sinh, tính bằng tuần. Đây là một biến liên tục.
Rectime	Tuổi thai tại thời điểm ghi dữ liệu (recording time), tính bằng tuần.
Age	Tuổi của mẹ tại thời điểm mang thai, tính theo năm.
Parity	Số lần mẹ đã sinh con sống trước đó. Ví dụ, 0 nếu đây là lần sinh đầu tiên.
Abortions	Số lần mẹ bị sảy thai hoặc phá thai trước đó.
Weight	Cân nặng của mẹ, thường tính bằng kg.
Hypertension	Mẹ có bị tăng huyết áp hay không. 0 = không, 1 = có.
Diabetes	Mẹ có bị tiểu đường hay không. 0 = không, 1 = có.
Placental_position	Vị trí nhau thai: ví dụ 1 = phía trước (anterior), 0 = phía sau (posterior).
Bleeding_first_trimester	Chảy máu trong 3 tháng đầu thai kỳ. 0 = không, 1 = có.
Bleeding_second_trimester	Chảy máu trong 3 tháng giữa thai kỳ. 0 = không, 1 = có.
Funneling	Funneling cổ tử cung (cervical funneling) – dấu hiệu nguy cơ sinh non. 0 = không, 1 = có.
Smoker	Mẹ có hút thuốc hay không. 0 = không, 1 = có.

In [28]:
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in df.columns:
    mode_val = df[col].mode()
    if len(mode_val) > 0:
        df[col] = df[col].fillna(mode_val[0])
    else:
        df[col] = df[col].fillna(0)



In [30]:
df.to_csv('data.csv', index=False, encoding='utf-8')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Rectime                    159 non-null    float64
 1   Age                        159 non-null    float64
 2   Parity                     159 non-null    float64
 3   Abortions                  159 non-null    float64
 4   Weight                     159 non-null    float64
 5   Bleeding_first_trimester   159 non-null    float64
 6   Bleeding_second_trimester  159 non-null    float64
 7   Smoker                     159 non-null    float64
dtypes: float64(8)
memory usage: 10.1 KB


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
df.to_csv('data.csv', index=False, encoding='utf-8')

In [ ]:
X_values=df.values
y_values=np.array(all_label)

In [ ]:
y_values.shape

In [ ]:
y_test_all = []
y_hat_all = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
for repeat in range(10):
    for train_index, test_index in kf.split(X_values):
        X_train, X_test = X_values[train_index], X_values[test_index]
        y_train, y_test = y_values[train_index], y_values[test_index]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        model = LogisticRegression(penalty='l1', solver='saga', max_iter=1000)
        model.fit(X_train_scaled, y_train)

        y_hat = model.predict_proba(X_test_scaled)[:, 1]
        

        y_test_all.extend(y_test)
        y_hat_all.extend(y_hat)

auc = roc_auc_score(y_test_all, y_hat_all)
print(f'Mean AUC over 20x5 folds: {auc:.4f}')